In [ ]:

import re
import time
import os
from IPython.display import clear_output
from selenium.webdriver.support.ui import Select

from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
def check_for_waiting_room(browser:
    
    html_source = browser.page_source

    # Check to see if there is a wait
    while ('wait time is' in html_source or 'greater than a day' in html_source or 
           html_source is None):
        html_source = browser.page_source
        time_remaining = re.findall('wait time is\n\s+(\d+) minute[s]*...', html_source)
        print('{}: There are {} minutes remaining'.format(count, time_remaining))
        
        time.sleep(2)
        clear_output(wait=True)
        
    os.system('say "Out of waiting room"')
    return html_source

In [ ]:
def check_for_available_appointments(browser, url=None):

    avail = None
    if url is not None:
        browser.get(url)
        
    html_source = browser.page_source

    if 'Clinic does not have any appointment slots available' in html_source:
        avail = False
    elif 'Please select a time for your appointment' not in html_source:
        avail = False
    else:
        available_times = re.findall("([0-9]+)\s+appointments available", html_source)
        if len(available_times)== 0:
            avail = False
        else:
            print('{} available'.format(len(available_times)))
            os.system('say "Available"')
            avail = True

    return avail, html_source


In [ ]:
urls = ['https://www.maimmunizations.org/client/registration?clinic_id=1510',
        'https://www.maimmunizations.org/client/registration?clinic_id=2483',
        'https://www.maimmunizations.org/client/registration?clinic_id=1511',
        'https://www.maimmunizations.org/client/registration?clinic_id=2484']

In [ ]:
# Open Chrome
browser = webdriver.Chrome('/Users/kathleennahabedian/Downloads/chromedriver')

# Enter the waiting room and wait for my turn
browser.get('https://www.maimmunizations.org/client/registration?clinic_id=1510')
html_source = check_for_waiting_room(browser)

In [ ]:
html_source = browser.page_source
do_break = False

while True:

    # If breaking out of the inner loop, break out of the outer loop (TODO: This is ugly, fix later)
    if do_break:
        break
        
    # Check each of the clinics to see if an appointment is available
    for url in urls:
        avail, html_source = check_for_available_appointments(browser, url)

        if avail:
            print(html_source)
            do_break = True
            break
        time.sleep(0.25)


# 1) Select appointment

In [ ]:
html_source = browser.page_source

# Split the page into all possible appointments
appts = html_source.split("consent-form-appointments")

# Loop over each appointment, figuring out which ones have numbers instead of "No appointments available"
avail_idx = []
for idx, appt in enumerate(appts):
    times = re.findall("([0-9]+)\s+appointments available", appt)
    if len(times) > 0:
        avail_idx.append(idx)
        
# Choose the last one (others will probably choose the first one? Improves chances of being the first to finish?)
appt = appts[avail_idx[-1]]
print(appt)

# Find the epoch time for this radio button, to identify it later
epoch = re.findall("value=\"([\d]+)\"", appt)[0]

# Click on this radio button
browser.find_element_by_xpath("//input[@value={}]".format(epoch)).click()

# Click save and continue
browser.find_element_by_id("submitButton").click()



# 2) Attestation

In [ ]:
# Wait for page to load
element = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, "patient_priority_group")))

# Choose eligibility group
selector = browser.find_element_by_xpath("//select[@class='form-select w-full']")
Select(selector).select_by_visible_text('Individuals age 65+')

# Check that you attest information is true
checkbox = browser.find_element_by_xpath("//input[@class='form-checkbox cursor-pointer']")
if not checkbox.get_attribute('checked'):
    checkbox.click()

# Click save and continue
browser.find_element_by_id("submitButton").click()

# 3) Personal info

In [ ]:
# Wait for page to load
element = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, "patient_ethnicity")))

# Enter personal info

browser.find_element_by_id("patient_first_name").clear()
browser.find_element_by_id("patient_first_name").send_keys('Bilbo')

browser.find_element_by_id("patient_last_name").clear()
browser.find_element_by_id("patient_last_name").send_keys('Baggins')

selector = browser.find_element_by_id("patient_race")
Select(selector).select_by_visible_text('Asian')

selector = browser.find_element_by_id("patient_ethnicity")
Select(selector).select_by_visible_text('Not Hispanic or Latino')

s = browser.find_element_by_id('patient_date_of_birth_2i')
Select(s).select_by_visible_text('January')

s = browser.find_element_by_id('patient_date_of_birth_3i')
Select(s).select_by_visible_text('1')

s = browser.find_element_by_id('patient_date_of_birth_1i')
Select(s).select_by_visible_text('1950')

s = browser.find_element_by_id('patient_sex')
Select(s).select_by_visible_text('Male')

browser.find_element_by_id("patient_email").clear()
browser.find_element_by_id("patient_email_confirmation").clear()
browser.find_element_by_id("patient_email").send_keys('bilbo@gmail.com')
browser.find_element_by_id("patient_email_confirmation").send_keys('bilbo@gmail.com')

browser.find_element_by_id("patient_phone_number").clear()
browser.find_element_by_id("patient_address").clear()
browser.find_element_by_id("locality").clear()
browser.find_element_by_id("patient_phone_number").send_keys('555-555-5555')
browser.find_element_by_id("patient_address").send_keys('123 Fake St')
browser.find_element_by_id("locality").send_keys('Wocester')

s = browser.find_element_by_id('administrative_area_level_1')
Select(s).select_by_visible_text('Massachusetts')

browser.find_element_by_id("postal_code").clear()
browser.find_element_by_id("postal_code").send_keys('01609')

# Click save and continue
browser.find_element_by_id("submitButton").click()

# 4) Insurance

In [ ]:
# Wait for page to load
element = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, "patient_insurance_company_name")))

# Insurance type
selector = browser.find_element_by_id("patient_insurance_type")
Select(selector).select_by_visible_text('Medicare')

# Insurance company
s = browser.find_element_by_id("patient_insurance_company_name")
Select(s).select_by_visible_text('BC/BS OF MASSACHUSETTS')

# Member id
browser.find_element_by_id("patient_member_id_for_insurance").clear()
browser.find_element_by_id("patient_member_id_for_insurance").send_keys('ABCD1234')

# First, last name
browser.find_element_by_id("patient_insured_first_name").clear()
browser.find_element_by_id("patient_insured_last_name").clear()
browser.find_element_by_id("patient_insured_first_name").send_keys('Bilbo')
browser.find_element_by_id("patient_insured_last_name").send_keys('Baggins')

# Birth date
s = browser.find_element_by_id('patient_insured_date_of_birth_2i')
Select(s).select_by_visible_text('January')

s = browser.find_element_by_id('patient_insured_date_of_birth_3i')
Select(s).select_by_visible_text('1')

s = browser.find_element_by_id('patient_insured_date_of_birth_1i')
Select(s).select_by_visible_text('1950')

# Policy holder relation
s = browser.find_element_by_id('patient_relation_to_patient_for_consent')
Select(s).select_by_visible_text('Self')

# Click save and continue
browser.find_element_by_id("submitButton").click()

# 5) Health questions

In [ ]:
# Wait for page to load
element = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, "7_no")))

# First or second dose?
browser.find_element_by_xpath("//input[@value='first_time']").click()

# Answer questions from health questionnaire
browser.find_element_by_id("7_no").click()
browser.find_element_by_id("8_no").click()
browser.find_element_by_id("9_no").click()
browser.find_element_by_id("10_no").click()
browser.find_element_by_id("11_no").click()
browser.find_element_by_id("12_no").click()
browser.find_element_by_id("13_no").click()
browser.find_element_by_id("14_no").click()
browser.find_element_by_id("15_no").click()
browser.find_element_by_id("16_no").click()
browser.find_element_by_id("17_no").click()

# Click save and continue
browser.find_element_by_id("skip_add_family").click()


# 6) Signature

In [ ]:
# Wait for page to load
element = WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.ID, "patient_vaccination_ids_30")))

# Select Moderna vaccine (the only option)
checkbox = browser.find_element_by_id("patient_vaccination_ids_30")
if not checkbox.get_attribute('checked'):
    checkbox.click()

# Relation to patient
s = browser.find_element_by_id('patient_relation_to_patient_for_insurance')
Select(s).select_by_visible_text('Self')

# Sign name
browser.find_element_by_id("patient_signer_first_name").clear()
browser.find_element_by_id("patient_signer_last_name").clear()
browser.find_element_by_id("patient_signer_first_name").send_keys('Bilbo')
browser.find_element_by_id("patient_signer_last_name").send_keys('Baggins')

# Click the blue box to sign name by typing
browser.find_element_by_xpath("//a[@class='text-xs font-bold uppercase px-5 py-3 cursor-pointer shadow-lg rounded block leading-normal bg-white text-blue-600']").click()

# Sign name
browser.find_element_by_id("patient_signatory_first_name").clear()
browser.find_element_by_id("patient_signatory_last_name").clear()
browser.find_element_by_id("patient_signatory_first_name").send_keys('Bilbo')
browser.find_element_by_id("patient_signatory_last_name").send_keys('Baggins')


# Click save and continue
browser.find_element_by_id("submitButton").click()

# 8) Review

In [ ]:

# Finalize registration submission
# TODO: Uncomment this? Or manually press submit in case things went awry?

#browser.find_element_by_id("submitButton").click()
#browser.find_element_by_xpath("//button[@value='appointment']").click()
